# DCGAN using Fashion-MNIST

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Reshape, Activation, Dense, Conv2D, Dropout
from keras.layers import Flatten, BatchNormalization, UpSampling2D, Conv2DTranspose
from keras.optimizers import RMSprop

Using TensorFlow backend.


### Dataset - Fashion MNIST

In [3]:
fashion_mnist = keras.datasets.fashion_mnist
(train_image, train_label), (test_image, test_label) = fashion_mnist.load_data()

In [24]:
train_image.shape

(60000, 28, 28)

In [25]:
train_image= train_image.reshape(-1, 28,28, 1).astype(np.float32)

In [26]:
train_image.shape

(60000, 28, 28, 1)

## Discriminator

In [5]:
def Disc():
    print('Hey, I am Discriminator. You can call me Disco.')
    depth = 64
    dropout = .4
    channel = 1
    #INPUT 28x28
    input_shape = (train_image[0].shape[0], train_image[0].shape[1], channel)
    Discriminator = Sequential()
    Discriminator.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,padding='same'))
    Discriminator.add(Activation('relu'))
    Discriminator.add(Dropout(dropout))
    Discriminator.add(Conv2D(depth*2, 5, strides=2, input_shape=input_shape,padding='same'))
    Discriminator.add(Activation('relu'))
    Discriminator.add(Dropout(dropout))
    Discriminator.add(Conv2D(depth*4, 5, strides=2, input_shape=input_shape,padding='same'))
    Discriminator.add(Activation('relu'))
    Discriminator.add(Dropout(dropout))
    Discriminator.add(Conv2D(depth*8, 5, strides=2, input_shape=input_shape,padding='same'))
    Discriminator.add(Activation('relu'))
    Discriminator.add(Dropout(dropout))
    Discriminator.add(Flatten())
    Discriminator.add(Dense(1))
    Discriminator.add(Activation('sigmoid'))
    Discriminator.summary()
    return Discriminator

## Generator

In [6]:
def Gen():
    print('Hey, I am Generator. You can call me Genko.')
    Generator =Sequential()
    dropout = 0.4
    depth = 64+64+64+64
    dim = 7
    Generator.add(Dense(dim*dim*depth, input_dim =100))
    Generator.add(BatchNormalization(momentum=0.9))
    Generator.add(Activation('relu'))
    Generator.add(Reshape((dim, dim, depth)))
    Generator.add(Dropout(dropout))
    #INPUT - dimxdimxdepth
    #Output - 2* dim x 2*dim x depth/2
    Generator.add(UpSampling2D())
    Generator.add(Conv2DTranspose(int(depth/2),5,padding ='same'))
    Generator.add(BatchNormalization(momentum=0.9))
    Generator.add(Activation('relu'))
    Generator.add(UpSampling2D())
    Generator.add(Conv2DTranspose(int(depth/4),5,padding ='same'))
    Generator.add(BatchNormalization(momentum=0.9))
    Generator.add(Activation('relu'))
    Generator.add(Conv2DTranspose(int(depth/8),5,padding ='same'))
    Generator.add(BatchNormalization(momentum=0.9))
    Generator.add(Activation('relu'))
    Generator.add(Conv2DTranspose(1,5, padding='same'))
    Generator.add(Activation('sigmoid'))
    Generator.summary()
    return Generator

### Discriminator Model

In [54]:
def Disc_M(noise, y):
    optimizer = RMSprop(lr=0.0008, clipvalue=1.0, decay=6e-8)
    Disc_model = Sequential([
        Disc()
    ])
    Disc_model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    return Disc_model.train_on_batch(noise, y)

## GAN full

In [55]:
def DCGAN(x,y):
    optimizer = RMSprop(lr=0.0004, clipvalue=1.0, decay=3e-8)
    GAN_model = Sequential([
        Gen(),
        Disc()
    ])
    GAN_model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    return GAN_model.train_on_batch(x, y)

## How to train your DCGAN

In [63]:
def train_dcgan(steps = 2000, batch_size = 256, save_interval=0):
    if save_interval>0:
            noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
    for i in range(steps):
        images_real = train_image[np.random.randint(0, train_image.shape[0], size=batch_size), :, :,:]
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
        images_fake = Gen().predict(noise)
        x = np.concatenate((images_real, images_fake))
        y = np.ones([2*batch_size, 1])
        y[batch_size:, :] = 0
        #d_loss = Disc_M().train_on_batch(x, y)
        d_loss = Disc_M(x,y)
        y = np.ones([batch_size, 1])
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100])
        #a_loss = DCGAN.train_on_batch(noise, y)
        a_loss = DCGAN(noise,y)
        log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
        log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
        print(log_mesg)
        if save_interval>0:
            if (i+1)%save_interval==0:
                self.plot_images(save2file=True, samples=noise_input.shape[0],\
                noise=noise_input, step=(i+1))

In [64]:
def plot_images(self, save2file=False, fake=True, samples=16, noise=None, step=0):
    filename = 'mnist.png'
    if fake:
        if noise is None:
            noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
        else:
            filename = "mnist_%d.png" % step
        images = Gen().predict(noise)
    else:
        i = np.random.randint(0, self.x_train.shape[0], samples)
        images = train_image[i, :, :, :]
    plt.figure(figsize=(10,10))
    for i in range(images.shape[0]):
        plt.subplot(4, 4, i+1)
        image = images[i, :, :, :]
        image = np.reshape(image, [self.img_rows, self.img_cols])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    if save2file:
        plt.savefig(filename)
        plt.close('all')
    else:
        plt.show()

In [ ]:
import time
start_time = time.time()

train_dcgan(steps=10000, batch_size=256, save_interval=500)


end_time = time.time()
elapsed = start_time-end_time
print('Seconds elapsed: ' +str(elapsed))

Hey, I am Generator. You can call me Genko.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_37 (Batc (None, 12544)             50176     
_________________________________________________________________
activation_81 (Activation)   (None, 12544)             0         
_________________________________________________________________
reshape_10 (Reshape)         (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_19 (UpSampling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_transpose_37 (Conv2DT (No

0: [D loss: 0.583082, acc: 0.480469]  [A loss: 0.692891, acc: 0.511719]
Hey, I am Generator. You can call me Genko.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_45 (Batc (None, 12544)             50176     
_________________________________________________________________
activation_101 (Activation)  (None, 12544)             0         
_________________________________________________________________
reshape_12 (Reshape)         (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_48 (Dropout)         (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_23 (UpSampling (None, 14, 14, 256)       0         
__________________________

1: [D loss: 1.337428, acc: 0.521484]  [A loss: 0.712219, acc: 0.105469]
Hey, I am Generator. You can call me Genko.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_53 (Batc (None, 12544)             50176     
_________________________________________________________________
activation_121 (Activation)  (None, 12544)             0         
_________________________________________________________________
reshape_14 (Reshape)         (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_58 (Dropout)         (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_27 (UpSampling (None, 14, 14, 256)       0         
__________________________

2: [D loss: 2.238424, acc: 0.478516]  [A loss: 0.692014, acc: 0.542969]
Hey, I am Generator. You can call me Genko.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_61 (Batc (None, 12544)             50176     
_________________________________________________________________
activation_141 (Activation)  (None, 12544)             0         
_________________________________________________________________
reshape_16 (Reshape)         (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_68 (Dropout)         (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_31 (UpSampling (None, 14, 14, 256)       0         
__________________________

conv2d_57 (Conv2D)           (None, 14, 14, 64)        1664      
_________________________________________________________________
activation_156 (Activation)  (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_74 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_58 (Conv2D)           (None, 7, 7, 128)         204928    
_________________________________________________________________
activation_157 (Activation)  (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_75 (Dropout)         (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 4, 4, 256)         819456    
_________________________________________________________________
activation_158 (Activation)  (None, 4, 4, 256)         0         
__________

...

44: [D loss: 1.335975, acc: 0.443359]  [A loss: 0.690347, acc: 0.617188]
Hey, I am Generator. You can call me Genko.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_197 (Dense)            (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_397 (Bat (None, 12544)             50176     
_________________________________________________________________
activation_981 (Activation)  (None, 12544)             0         
_________________________________________________________________
reshape_100 (Reshape)        (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_488 (Dropout)        (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_199 (UpSamplin (None, 14, 14, 256)       0         
_________________________

45: [D loss: 1.071000, acc: 0.494141]  [A loss: 0.697390, acc: 0.304688]
Hey, I am Generator. You can call me Genko.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_201 (Dense)            (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_405 (Bat (None, 12544)             50176     
_________________________________________________________________
activation_1001 (Activation) (None, 12544)             0         
_________________________________________________________________
reshape_102 (Reshape)        (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_498 (Dropout)        (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_203 (UpSamplin (None, 14, 14, 256)       0         
_________________________

46: [D loss: 1.299885, acc: 0.476562]  [A loss: 0.699100, acc: 0.328125]
Hey, I am Generator. You can call me Genko.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_205 (Dense)            (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_413 (Bat (None, 12544)             50176     
_________________________________________________________________
activation_1021 (Activation) (None, 12544)             0         
_________________________________________________________________
reshape_104 (Reshape)        (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_508 (Dropout)        (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_207 (UpSamplin (None, 14, 14, 256)       0         
_________________________

conv2d_412 (Conv2D)          (None, 2, 2, 512)         3277312   
_________________________________________________________________
activation_1039 (Activation) (None, 2, 2, 512)         0         
_________________________________________________________________
dropout_517 (Dropout)        (None, 2, 2, 512)         0         
_________________________________________________________________
flatten_103 (Flatten)        (None, 2048)              0         
_________________________________________________________________
dense_208 (Dense)            (None, 1)                 2049      
_________________________________________________________________
activation_1040 (Activation) (None, 1)                 0         
Total params: 4,305,409
Trainable params: 4,305,409
Non-trainable params: 0
_________________________________________________________________
47: [D loss: 3.258931, acc: 0.501953]  [A loss: 0.691395, acc: 0.570312]
Hey, I am Generator. You can call me Genko.
_______________

Layer (type)                 Output Shape              Param #   
conv2d_417 (Conv2D)          (None, 14, 14, 64)        1664      
_________________________________________________________________
activation_1056 (Activation) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_524 (Dropout)        (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_418 (Conv2D)          (None, 7, 7, 128)         204928    
_________________________________________________________________
activation_1057 (Activation) (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_525 (Dropout)        (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_419 (Conv2D)          (None, 4, 4, 256)         819456    
_________________________________________________________________
activation